# Yada Deconvolution
---

- Run the following cells for deconvolution using Yada.

## 1 - Import Prerequisites.

In [ ]:
#On Colab.
!pip install -q tslearn gseapy similaritymeasures
!git clone https://github.com/zurkin1/Yada.git
!mv Yada/* .

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import FileLink, FileLinks
import pandas as pd
from yada import *

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)

## 2 - Configure Input Files.

Example input files are in the ./data/ folder.


In [55]:
#This is the mixture file in the format: columns: mix1, mix2, ..., rows: gene names.
mix = './data/Challenge/mix-107019_RNASeq.csv'

#Reference matrix name. Should be normalized as the mix data.
pure = './data/Challenge/pure-107019_RNASeq.csv'

#True cell type proportions file.
labels = './data/Challenge/prop-107019_RNASeq.csv'

## 3 - Run Deconvolution.

In [17]:
mix, pure, gene_list_df = preprocess(mix, pure)
result = run_dtw_deconv(mix, pure, gene_list_df)
result

#Download Result.
#FileLink('data/results.csv')
#from google.colab import files
#files.download('data/results.csv') 

,naive.B.cells,memory.B.cells,naive.CD4.T.cells,naive.CD8.T.cells,memory.CD8.T.cells,regulatory.T.cells,monocytes,NK.cells,myeloid.dendritic.cells,neutrophils
mix0,0.000000,2.411721e-01,0.000000e+00,0.000000e+00,8.721292e-02,1.654319e-01,1.497608e-01,1.821413e-02,1.503610e-01,0.187847
mix1,0.150872,4.812451e-02,3.475842e-11,0.000000e+00,0.000000e+00,1.723071e-01,1.845105e-01,1.111955e-01,3.244170e-01,0.008573
mix2,0.066226,1.685786e-01,6.363351e-02,7.111752e-02,2.308617e-01,6.838576e-02,8.508581e-02,3.104875e-02,3.348154e-02,0.181580
mix3,0.307865,0.000000e+00,1.230014e-01,1.449526e-01,0.000000e+00,0.000000e+00,1.824844e-01,1.204828e-01,0.000000e+00,0.125045
mix4,0.115112,6.118550e-15,3.628799e-15,2.099705e-02,1.270943e-01,1.298876e-01,2.718646e-01,2.826728e-01,5.131808e-02,0.001053
mix5,0.316086,2.227409e-02,7.592709e-10,2.095167e-01,1.612603e-02,1.427691e-01,0.000000e+00,1.334211e-02,2.040157e-01,0.075870
mix6,0.076283,2.577672e-02,1.031090e-01,6.546050e-14,6.159943e-02,1.079747e-12,3.814416e-01,3.114390e-01,1.065416e-02,0.029698
mix7,0.274009,6.965705e-02,2.306733e-01,1.430616e-01,1.502480e-12,1.279961e-01,5.270309e-02,0.000000e+00,1.124479e-02,0.090655
mix8,0.059322,2.217562e-01,3.113964e-02,1.509374e-01,3.168889e-14,3.336183e-01,0.000000e+00,3.325314e-02,1.400284e-01,0.029945
mix9,0.134989,5.041417e-02,2.051428e-01,9.395370e-02,1.083234e-03,5.958823e-02,1.674474e-01,1.854487e-15,2.171078e-01,0.070273


## 4 - Evaluate Results.

In case true proportions are available.

In [57]:
calc_corr(labels, result)

,cell type,Pearson,Spearman,P
0,naive.B.cells,0.884305,0.879699,3.213670e-07
1,memory.B.cells,0.899984,0.834338,4.810522e-06
2,naive.CD4.T.cells,0.810546,0.777109,5.555559e-05
3,naive.CD8.T.cells,0.582671,0.600904,5.078524e-03
4,memory.CD8.T.cells,0.845725,0.487551,2.921517e-02
5,regulatory.T.cells,0.655329,0.557895,1.058503e-02
6,monocytes,0.956189,0.919143,1.043339e-08
7,NK.cells,0.983158,0.930827,2.674426e-09
8,myeloid.dendritic.cells,0.890522,0.881203,2.886085e-07
9,neutrophils,0.970342,0.948872,1.882170e-10


## 5 - Benchmark.

In [113]:
data = pd.read_csv('./data/result.csv', index_col=0)
data.columns = ['metric', 'dataset', 'platform', 'celltype', 'pearson', 'spearman', 'p']
data = data.drop(['pearson', 'p'], axis=1)
data = data.groupby(['metric', 'dataset']).mean().reset_index().pivot(index='dataset', columns='metric', values='spearman').reset_index()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


plt.rcParams["figure.figsize"] = [20,20]

# Make a data frame
#df=pd.DataFrame({'x': range(1,11), 'y1': np.random.randn(10), 'y2': np.random.randn(10)+range(1,11), 'y3': np.random.randn(10)+range(11,21), 'y4': np.random.randn(10)+range(6,16), 'y5': np.random.randn(10)+range(4,14)+(0,0,0,0,0,0,0,-3,-8,-6), 'y6': np.random.randn(10)+range(2,12), 'y7': np.random.randn(10)+range(5,15), 'y8': np.random.randn(10)+range(4,14), 'y9': np.random.randn(10)+range(4,14) })
df = pd.read_csv('./data/result.csv', index_col=0)
df.columns = ['metric', 'dataset', 'platform', 'celltype', 'pearson', 'spearman', 'p']
df = df.drop(['pearson', 'p'], axis=1)
df = df.groupby(['metric', 'dataset']).mean().reset_index().pivot(index='dataset', columns='metric', values='spearman').reset_index()

# Initialize the figure style
plt.style.use('seaborn-darkgrid')
 
# create a color palette
palette = plt.get_cmap('Set1')
 
# multiple line plot
num=0
for column in df.drop('dataset', axis=1):
    num+=1
    
    df = df.sort_values(column, ascending=False)
    # Find the right spot on the plot
    plt.subplot(3,3, num)
 
    # plot every group, but discrete
    for v in df.drop('dataset', axis=1):
        plt.plot(df['dataset'], df[v], marker='', color='grey', linewidth=0.6, alpha=0.3)
 
    # Plot the lineplot
    plt.plot(df['dataset'], df[column], marker='', color=palette(num), linewidth=2.4, alpha=0.9, label=column)
 
    # Same limits for every chart
    plt.xlim(0,10)
    plt.ylim(0,1)
 
    # Not ticks everywhere
    if num in range(7) :
        plt.tick_params(labelbottom='off')
    if num not in [1,4,7] :
        plt.tick_params(labelleft='off')
 
    # Add title
    plt.title(f'{column} ({np.round(df[column].mean(), 2)})', loc='left', fontsize=16, fontweight=0, color=palette(num) )
    plt.xticks(rotation=45)

# general title
plt.suptitle("Spearman correlation of different metrics over ten datasets", fontsize=13, fontweight=0, color='black', style='italic', y=1.02)
 
# Axis titles
#plt.text(0.5, 0.02, 'Time', ha='center', va='center')
#plt.text(0.06, 0.5, 'Note', ha='center', va='center', rotation='vertical')

# Show the graph
plt.show()